### Przedmiot: Python 2
### Temat: "Deskryptory / Dekoratory / Właściwości"
### Imię nazwisko: Eduard Povierin
### Grupa: 2TD12A

---

In [4]:
# 1.
class MyDescriptor:
    def __init__(self, init_val=None, name='var'):
        self.val = init_val
        self.name = name
        
    def __get__(self, obj, obj_type):
        print('Retrieving', self.name)
        try:
            return self.val
        except:
            print('Variable is not declared')
            
    def __set__(self, obj, val):
        print('Updating', self.name)
        self.val = val
        
    def __delete__(self, obj):
        print('Deleting', self.name)
        try:
            del self.val
        except:
            print('Variable is not declared')
            
            
class MyClass:
     x = MyDescriptor(10, 'variable x')
     y = 5
     
     
m = MyClass()
print(m.x)
m.x = 20

print(m.x)
print(m.y)

del m.x
print(m.x)

m.x = 4
print(m.x)

Retrieving variable x
10
Updating variable x
Retrieving variable x
20
5
Deleting variable x
Retrieving variable x
Variable is not declared
None
Updating variable x
Retrieving variable x
4


In [10]:
# 2.
class AgeDescriptor:
     def __get__(self, obj, objtype):
          return obj._age
     
     def __set__(self, obj, age):
          if not isinstance(age, int):
               raise TypeError("Age must be an integer.")
          if age < 0 or age > 120:
               raise ValueError("Age must be between 0 and 120.")
          obj._age = age

class Person:
     def __init__(self, name, age):
          self.name = name
          self.age = age
          
     age = AgeDescriptor()

p = Person("Alice", 25)
print(p.age)

p.age = 30
print(p.age)

# p.age = "thirty" - Powoduje błąd

25
30


In [13]:
# 3.
class FuelCapacityDescriptor:
    def __init__(self):
        self.__fuel_cap = 0
        
    def __get__(self, instance, owner):
        return self.__fuel_cap
    
    def __set__(self, instance, value):
        if isinstance(value, int):
            if value < 0:
                raise ValueError("Fuel Capacity can never be less than zero")
            else:
                self.__fuel_cap = value
        else:
            raise TypeError("Fuel Capacity can only be an integer")
            
    def __delete__(self, instance):
        del self.__fuel_cap
        
        
class Car:
    fuel_cap = FuelCapacityDescriptor()
    
    def __init__(self, make, model, fuel_cap):
        self.make = make
        self.model = model
        self.fuel_cap = fuel_cap
        self.fuel_level = fuel_cap
        
    def __str__(self):
        return "{0} model {1} with a fuel capacity of {2} ltr.".format(self.make, self.model, self.fuel_cap)
    
    def needs_refuel(self):
        if self.fuel_level < self.fuel_cap * 0.2:
            return True
        else:
            return False
        
    def refuel(self, amount):
        if amount < 0:
            raise ValueError("Fuel amount can never be less than zero")
        elif amount == 0:
            print("No fuel added")
        else:
            self.fuel_level += amount
            if self.fuel_level > self.fuel_cap:
                self.fuel_level = self.fuel_cap
            print("Added {0} liters of fuel. Current fuel level: {1} liters".format(amount, self.fuel_level))
        
car2 = Car("BMW", "X7", 40)
print(car2)

print("Car needs refuel: ", car2.needs_refuel())
car2.fuel_level = 5

print("Car needs refuel: ", car2.needs_refuel())
car2.refuel(10)
print(car2.needs_refuel())


BMW model X7 with a fuel capacity of 40 ltr.
Car needs refuel:  False
Car needs refuel:  True
Added 10 liters of fuel. Current fuel level: 15 liters
False


In [16]:
# 4.
import time

def timer_decorator(func):
    def wrapper():
        start_time = time.time()
        func()
        end_time = time.time()
        print("Time taken: ", end_time - start_time)
    return wrapper

@timer_decorator
def say_whee():
    print("Whee!")

say_whee()


Whee!
Time taken:  0.00014495849609375


In [25]:
# 5. 
import time

def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        start_time = time.time()
        func()
        end_time = time.time()
        print("Something is happening after the function is called.")
        print("Time taken: ", end_time - start_time)
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.
Time taken:  5.7220458984375e-06


In [27]:
# 6.
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def my_print(text):
    print(text)

@do_twice
def my_sum(a, b):
    print(a + b)

my_print('test')
my_sum(2, 3)


test
test
5
5


In [34]:
# 7.
from functools import wraps
import time

def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        start_time = time.time()
        result = f(*args, **kwds)
        end_time = time.time()
        print('Time taken:', end_time - start_time)
        return result
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')

example()
print(example.__name__)
print(example.__doc__)


Calling decorated function
Called example function
Time taken: 1.0013580322265625e-05
example
Docstring


In [35]:
# 8.
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        value = func(*args, **kwds)
        print('End of decorated function call')
        return value
    return wrapper

@my_decorator
def example():
    """This is the documentation of the function"""
    print('This is a decorated function')

example()
print(example.__name__)
print(example.__doc__)


Calling decorated function
This is a decorated function
End of decorated function call
example
This is the documentation of the function


In [41]:
# 9.
from functools import wraps
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])
        time.sleep(0.1)

waste_some_time(5)

Finished 'waste_some_time' in 0.5123 secs


In [42]:
# 10.
class Person(object):
    def __init__(self):
        self._name = ''
        self._age = 0
        
    def name_getter(self):
        print("Getting name: %s" % self._name)
        return self._name
    
    def name_setter(self, value):
        print("Setting name: %s" % value)
        self._name = value.title()
        
    def name_deleter(self):
        print("Deleting name: %s" % self._name)
        del self._name
        
    def age_getter(self):
        print("Getting age: %d" % self._age)
        return self._age
    
    def age_setter(self, value):
        print("Setting age: %d" % value)
        self._age = value
        
    def age_deleter(self):
        print("Deleting age: %d" % self._age)
        del self._age
        
    name = property(name_getter, name_setter, name_deleter, "I'm the name property.")
    age = property(age_getter, age_setter, age_deleter, "I'm the age property.")
    
user = Person()
user.name = 'john smith'
user.age = 25
print(user.name)
print(user.age)
del user.name
del user.age


Setting name: john smith
Setting age: 25
Getting name: John Smith
John Smith
Getting age: 25
25
Deleting name: John Smith
Deleting age: 25


In [43]:
# 11.
# Alphabet class
class Alphabet:
    def __init__(self, value):
        self._value = value
    # getting the values
    def get_value(self):
        print('Getting value')
        return self._value
    # setting the values
    def set_value(self, value):
        print('Setting value to ' + value)
        self._value = value
    # deleting the values
    def del_value(self):
        print('Deleting value')
        del self._value
    value = property(get_value, set_value, del_value, )
    
# create an instance of the Alphabet class
x = Alphabet('Hello World')

# print the value of x
print(x.value)

# set the value of x
x.value = 'Goodbye World'

# print the new value of x
print(x.value)

# delete the value of x
del x.value

Getting value
Hello World
Setting value to Goodbye World
Getting value
Goodbye World
Deleting value


In [44]:
# 12.
class Person:
    def __init__(self):
        self._name = ''
        self._age = 0
        
    @property
    def name(self):
        print("Getting name: %s" % self._name)
        return self._name
    
    @name.setter
    def name(self, value):
        print("Setting name: %s" % value)
        self._name = value.title()
        
    @name.deleter
    def name(self):
        print("Deleting name: %s" % self._name)
        del self._name
        
    @property
    def age(self):
        print("Getting age: %d" % self._age)
        return self._age
    
    @age.setter
    def age(self, value):
        print("Setting age: %d" % value)
        self._age = value
        
    @age.deleter
    def age(self):
        print("Deleting age: %d" % self._age)
        del self._age
        
user = Person()
user.name = 'john smith'
user.age = 25
print(user.name)
print(user.age)
del user.name
del user.age


Setting name: john smith
Setting age: 25
Getting name: John Smith
John Smith
Getting age: 25
25
Deleting name: John Smith
Deleting age: 25


In [46]:
# 13.
class Alphabet:
    def __init__(self, value):
        self._value = value
    
    @property
    def value(self):
        print('Getting value')
        return self._value
    
    @value.setter
    def value(self, value):
        print('Setting value to ' + value)
        self._value = value.title() 
    
    @value.deleter
    def value(self):
        print('Deleting value')
        del self._value

x = Alphabet('hello world')

print(x.value)

x.value = 'goodbye world'

print(x.value)

del x.value

Getting value
hello world
Setting value to goodbye world
Getting value
Goodbye World
Deleting value
